In [5]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
import json

In [2]:
from utility import get_spotify_client, get_spotify_oauth

In [4]:
%%capture
sp = get_spotify_oauth()

In [40]:
with open('similarity_ids.json', 'r') as f:
    json_obj = json.load(f)
my_id = json_obj['trystan']
their_id = json_obj['josiah']

In [83]:
def get_users_tracks_from_playlists(user_id, limit=15, offset=0):
    playlist_df = pd.DataFrame.from_records(sp.user_playlists(user_id, limit=limit, offset=offset)['items'])[['id', 'name', 'uri']]
    return pd.concat([pd.DataFrame.from_records([jo['track'] for jo in sp.playlist_tracks(playlist_id)['items']])[['id', 'name', 'uri']] for playlist_id in playlist_df['id']])

In [106]:
# Good idea to make sure there's about the same number of tracks in each dataframe.
my_tracks_df = get_users_tracks_from_playlists(my_id, limit=40)
my_tracks_df

,id,name,uri
0,6StYvRLBf8wZPng4WO6buh,Messy,spotify:track:6StYvRLBf8wZPng4WO6buh
1,6zdTpSc756yforxfthMKd2,My Future Ex Wife,spotify:track:6zdTpSc756yforxfthMKd2
2,7qJ0B5vtFGzXwQ6b18C9ri,Right Through Me,spotify:track:7qJ0B5vtFGzXwQ6b18C9ri
3,2pWjgVom86aF5F0JKAq4h0,Constance,spotify:track:2pWjgVom86aF5F0JKAq4h0
4,0VwhP56nyBybH9WhLhMPrG,Mr. Magic,spotify:track:0VwhP56nyBybH9WhLhMPrG
...,...,...,...
11,6PbUDvtc6Oom5pRbPnxKz2,Crystal Tokyo,spotify:track:6PbUDvtc6Oom5pRbPnxKz2
12,56oi3AuCS4WEv4RuGDE0Uy,Delete - Bossfight Remix,spotify:track:56oi3AuCS4WEv4RuGDE0Uy
13,4KwOmxzcf2zcFUmgo6CBTR,she speaks,spotify:track:4KwOmxzcf2zcFUmgo6CBTR
14,1ODEuo1QE9alDUNPOEcUBi,Velocity,spotify:track:1ODEuo1QE9alDUNPOEcUBi


In [105]:
their_tracks_df = get_users_tracks_from_playlists(their_id, limit=32)
their_tracks_df

,id,name,uri
0,6pooRNiLyYpxZeIA5kJ5EX,Good Things Fall Apart (with Jon Bellion),spotify:track:6pooRNiLyYpxZeIA5kJ5EX
1,2Qp4AzHWghlRC2VAWmzgHB,Green Eyes,spotify:track:2Qp4AzHWghlRC2VAWmzgHB
2,169pWCmfvVazzcL4EXFPgL,You Know It,spotify:track:169pWCmfvVazzcL4EXFPgL
3,0yLbdeni9HurhZJOyvmD13,I Want It All,spotify:track:0yLbdeni9HurhZJOyvmD13
4,5eebwQwrOQN5rZgR05HGef,Mason-Dixon Line,spotify:track:5eebwQwrOQN5rZgR05HGef
...,...,...,...
41,7KjmxWkC15gNyeUskvp1fv,What Child Is This,spotify:track:7KjmxWkC15gNyeUskvp1fv
42,2WsfrJof0Hh9fz0ZxTC2Ey,Silent Night,spotify:track:2WsfrJof0Hh9fz0ZxTC2Ey
43,1B4B033tgLq9ifm2st6djz,Peace on Earth,spotify:track:1B4B033tgLq9ifm2st6djz
44,3ak66id2FyW67R6eB1nxfY,Pass Me A Pint,spotify:track:3ak66id2FyW67R6eB1nxfY


In [101]:
def get_recommendations(tracks_df: pd.DataFrame):
    most_common_tracks = [t[0] for t in tracks_df.value_counts(subset=['id']).index.tolist()[:5]]
    return pd.DataFrame.from_records(sp.recommendations(country='US', seed_tracks=most_common_tracks)['tracks'])[['id', 'name', 'uri']]

In [110]:
my_recommendations_df = get_recommendations(my_tracks_df)
my_recommendations_df

,id,name,uri
0,0T03u6F5NcBWuWMsZYCAXY,Monolith,spotify:track:0T03u6F5NcBWuWMsZYCAXY
1,2jXhm9rDyfoKKgTffUYqmw,Okayalright,spotify:track:2jXhm9rDyfoKKgTffUYqmw
2,3fSdTVHwb8ZyiYAFSE00qd,Deeper Deeper,spotify:track:3fSdTVHwb8ZyiYAFSE00qd
3,0G6cDf9GNtv0ZkJNAMFLrZ,Yoshu Fukushu,spotify:track:0G6cDf9GNtv0ZkJNAMFLrZ
4,1p6ec2fyWKl0cpuEtrjXel,El Pueblo Unido,spotify:track:1p6ec2fyWKl0cpuEtrjXel
5,10TVQRxEOS3LEIMmSWsRZq,MC Thunder II (Dancing Like a Ninja),spotify:track:10TVQRxEOS3LEIMmSWsRZq
6,4N98sfqOlMlP1iO7kuW0t9,RAISE MY SWORD,spotify:track:4N98sfqOlMlP1iO7kuW0t9
7,30CenAPBR2VmH91r4Um32b,Clock Strikes,spotify:track:30CenAPBR2VmH91r4Um32b
8,3vLsiXpez6NhD97FUMntKL,Black Heart (Re-Recorded),spotify:track:3vLsiXpez6NhD97FUMntKL
9,3dVMeI08Of33bjSELmtSqq,"1,000,000 TIMES feat. chelly (EGOIST)",spotify:track:3dVMeI08Of33bjSELmtSqq


In [109]:
their_recommendations_df = get_recommendations(their_tracks_df)
their_recommendations_df

,id,name,uri
0,22jEGuVPtvtMqKPuXdOVCh,SAD (Clap Your Hands),spotify:track:22jEGuVPtvtMqKPuXdOVCh
1,4lo3bVqV0U805BdS1Pagki,Shotgun - Acoustic Version,spotify:track:4lo3bVqV0U805BdS1Pagki
2,5fIq2ovEhDIyV5Hx1epC4c,1000 Nights,spotify:track:5fIq2ovEhDIyV5Hx1epC4c
3,2EWpa5XnAuSn0sIkSSIhYk,Wish I Knew You,spotify:track:2EWpa5XnAuSn0sIkSSIhYk
4,0OlS52TmtK2gPaEtdmr7DN,No Such Thing as a Broken Heart,spotify:track:0OlS52TmtK2gPaEtdmr7DN
5,5OUSPcqhYTOzpbXzoEvKim,The Fighter,spotify:track:5OUSPcqhYTOzpbXzoEvKim
6,0VcVy0kpfwyd5rMS5URyVD,Joyride,spotify:track:0VcVy0kpfwyd5rMS5URyVD
7,7zJg7aNCvTKW9EtG1Dvzkl,Fix My Eyes,spotify:track:7zJg7aNCvTKW9EtG1Dvzkl
8,6p4mASLulKdpNFLxQhs0ff,Real Peach,spotify:track:6p4mASLulKdpNFLxQhs0ff
9,74HX1HcsR135apNYpHUUZj,That Was A Crazy Game Of Poker,spotify:track:74HX1HcsR135apNYpHUUZj
